# Chinak Analysis

Final Data years
- 2016
- 2015

Preliminary Data years
- 2017+


In [3]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import datetime
import glob

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import YearLocator, WeekdayLocator, MonthLocator, DayLocator, HourLocator, DateFormatter

import cmocean

server_url = 'http://downdraft.pmel.noaa.gov:8080/erddap'

In [5]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for='Mooring AND cb'))

print(df['Dataset ID'].values)

['datasets_Mooring_15cb1a_preliminary' 'datasets_Mooring_16cb1a_final'
 'datasets_Mooring_16cb1a_preliminary'
 'datasets_Mooring_17cb1a_preliminary' 'datasets_Mooring_10cb1a_final'
 'datasets_Mooring_15cb1a_final' 'datasets_Mooring_04cb1a_final'
 'datasets_Mooring_05cb1a_final' 'datasets_Mooring_06cb1a_final'
 'datasets_Mooring_07cb1a_final' 'datasets_Mooring_08cb1a_final'
 'datasets_Mooring_09cb1a_final' 'datasets_Mooring_11cb1a_final'
 'datasets_Mooring_11cb1b_final' 'datasets_Mooring_12cb1b_final'
 'datasets_Mooring_12cb1a_final' 'datasets_Mooring_13cb1a_final'
 'CTD_mf1006_final']


Get **Temperature** only (and depth/time)

In [6]:
from requests.exceptions import HTTPError

dfs = {}
for dataset_id in df['Dataset ID'].values:
    print(dataset_id)
    try:
        d = ERDDAP(server=server_url,
            protocol='tabledap',
            response='csv'
        )
        d.dataset_id=dataset_id
        d.variables = ['time','depth','temperature']
        d.constraints
    except HTTPError:
        print('Failed to generate url {}'.format(dataset_id))

    try:
        df_m = d.to_pandas(
                    index_col='time (UTC)',
                    parse_dates=True,
                    skiprows=(1,)  # units information can be dropped.
                    )
        df_m.sort_index(inplace=True)
        df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]
        
        #-9999 and 1e35 are missing values... but erddap doesn't catch the -9999 yet 
        #and some 1e35 are off a bit
        df_m['depth'][(df_m.depth == -9999)] = np.nan # only in pressure field
        dfs.update({dataset_id:df_m})
    except:
        pass


datasets_Mooring_15cb1a_preliminary
datasets_Mooring_16cb1a_final
datasets_Mooring_16cb1a_preliminary
datasets_Mooring_17cb1a_preliminary
datasets_Mooring_10cb1a_final
datasets_Mooring_15cb1a_final
datasets_Mooring_04cb1a_final
datasets_Mooring_05cb1a_final
datasets_Mooring_06cb1a_final
datasets_Mooring_07cb1a_final
datasets_Mooring_08cb1a_final
datasets_Mooring_09cb1a_final
datasets_Mooring_11cb1a_final
datasets_Mooring_11cb1b_final
datasets_Mooring_12cb1b_final
datasets_Mooring_12cb1a_final
datasets_Mooring_13cb1a_final
CTD_mf1006_final


In [50]:
for df in dfs:
    if df in ['datasets_Mooring_16cb1a_final','datasets_Mooring_17cb1a_preliminary']:
        dfg = dfs[df].groupby('depth').groups
        deepestInd = np.array(list(dfg.keys())).max()
        dfm = dfs[df][dfs[df].depth == deepestInd].resample('M').mean()
        print(f"evaluated at {deepestInd} for {df}")
        print(dfm)

evaluated at 188.0 for datasets_Mooring_16cb1a_final
                           depth  temperature
time (UTC)                                   
2016-02-29 00:00:00+00:00  188.0     5.709098
2016-03-31 00:00:00+00:00  188.0     5.712677
2016-04-30 00:00:00+00:00  188.0     5.980544
2016-05-31 00:00:00+00:00  188.0     6.826572
2016-06-30 00:00:00+00:00  188.0     7.705046
2016-07-31 00:00:00+00:00  188.0     8.595781
2016-08-31 00:00:00+00:00  188.0     9.016684
2016-09-30 00:00:00+00:00  188.0     8.958499
2016-10-31 00:00:00+00:00  188.0     8.742800
2016-11-30 00:00:00+00:00  188.0     7.854665
2016-12-31 00:00:00+00:00  188.0     6.308898
2017-01-31 00:00:00+00:00  188.0     5.038120
2017-02-28 00:00:00+00:00  188.0     4.706823
2017-03-31 00:00:00+00:00  188.0     3.409792
2017-04-30 00:00:00+00:00  188.0     3.693339
evaluated at 188.0 for datasets_Mooring_17cb1a_preliminary
                           depth  temperature
time (UTC)                                   
2017-04-30 00:

In [44]:
dfs[df].depth == deepestInd

time (UTC)
2016-02-09 21:00:00+00:00     True
2016-02-09 22:00:00+00:00     True
2016-02-09 22:00:00+00:00    False
2016-02-09 23:00:00+00:00    False
2016-02-09 23:00:00+00:00     True
                             ...  
2017-04-22 22:00:00+00:00     True
2017-04-22 23:00:00+00:00     True
2017-04-23 00:00:00+00:00     True
2017-04-23 01:00:00+00:00     True
2017-04-23 02:00:00+00:00     True
Name: depth, Length: 18117, dtype: bool